In [1]:
from collections import defaultdict
from heapq import heappop, heappush
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d

In [2]:
with open('../data/2024/day20.txt') as f:
    data = f.read()

In [3]:
grid = grid2d.parse(data)
grid_extents = max(grid)
start = next(iter(grid2d.find_coords(grid, 'S')))
goal = next(iter(grid2d.find_coords(grid, 'E')))
grid[start], grid[goal] = '.', '.'

In [4]:
def dijkstra(coord: tuple) -> tuple:
    cost_from_start = defaultdict(lambda: float('inf'))
    prev_node = defaultdict(None)
    queue = [(0, coord)]

    while queue:
        cost, at = heappop(queue)
        if at == goal:
            # Rebuild path
            path, i = [], at
            while True:
                if i is None or i == start: break
                path.append(i)
                i = prev_node[i]

            return cost, list(path + [start])

        for n_at, tile in grid2d.neighbors(grid, at, vector2d.NESW):
            if tile == '#': continue
            new_cost = cost + 1

            if new_cost < cost_from_start[n_at]:
                cost_from_start[n_at] = new_cost
                prev_node[n_at] = at
                heappush(queue, (new_cost, n_at))
    return None, None

In [5]:
# Compute the shortest (only) path and the time remaining from each step
best_t, best_path = dijkstra(start)
t_left_from_coord = defaultdict(None, {yx:cost for cost, yx in enumerate(best_path)})

In [6]:
# Part 1
# A shortcut to find the best cheats is to look at all inner walls
# and see if they connect two straight-line walkable tiles with a
# delta greater than the goal.
# 20<wall>10 skips 8 steps going right (10 less 2 for ->wall->10)
best_cheats = 0
BEST_CHEAT_THRESHOLD = 100

for yx in grid2d.fill((1,1), (grid_extents[0]-1, grid_extents[1]-1)):
    tile = grid[yx]
    if tile != '#': continue

    n,e,s,w = vector2d.north(yx), vector2d.east(yx), vector2d.south(yx), vector2d.west(yx)
    if t_left_from_coord.get(w) is not None and t_left_from_coord.get(e) is not None:
        delta_we = t_left_from_coord[w] - t_left_from_coord[e]
        if abs(delta_we) - 2 >= BEST_CHEAT_THRESHOLD:
            best_cheats += 1
    elif t_left_from_coord.get(n) is not None and t_left_from_coord.get(s) is not None:
        delta_ns = t_left_from_coord[n] - t_left_from_coord[s]
        if abs(delta_ns) - 2 >= BEST_CHEAT_THRESHOLD:
            best_cheats += 1

print("Part 1:", best_cheats)

Part 1: 1393
